In [103]:
import pandas as pd
from fuzzywuzzy import fuzz
import recordlinkage
imdb = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/IMDB_clean.csv')
omdb = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/Omdb_clean.csv')
mojo = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/mojo_clean.csv')
rotten = pd.read_csv('/Users/Amin/Documents/GitHub/Movie_boxoffice_reviews/data/interim/Rotten_clean.csv')
rotten = rotten.rename(columns = {'movie_title':'Title'})
mojo = mojo.rename(columns = {'title':'Title'})
rotten['runtime']=rotten['runtime'].astype(int)


df = imdb.merge(omdb,left_on='imdb_title_id',right_on='imdbID',how='inner')

df.drop(columns=['Unnamed: 0_y','imdbID','Released','Unnamed: 0_x','Director','Year'],inplace=True)
df.head()
col = df.columns.tolist()
new_col=['imdb_title_id','Title','title','director','year','date_published' ,'Runtime','avg_vote','total_votes',
         'us_voters_rating','us_voters_votes','non_us_voters_rating','non_us_voters_votes','females_allages_avg_vote',
         'females_allages_votes','males_allages_avg_vote','males_allages_votes','Metascore']
df = df[new_col]
df = df.rename(columns={'title':'Alternative Title','avg_vote':'imdb_avg_vote','total_votes':'imdb_total_votes',
                       'us_voters_rating':'imdb_us_voters_rating','us_voters_votes':'imdb_us_voters_votes',
                       'non_us_voters_rating':'imdb_non_us_voters_rating','non_us_voters_votes':'imdb_non_us_voters_votes',
                       'females_allages_avg_vote':'imdb_females__avg_vote','females_allages_votes':'imdb_females_votes',
                       'males_allages_avg_vote':'imdb_males_avg_vote','males_allages_votes':'imdb_males_votes'})
df['Runtime'] = df['Runtime'].str.replace(' min','')
df['Runtime'] = df['Runtime'].astype(int)
#print(df.head())
#print(df.shape)
df1 = df.merge(rotten,left_on='Title',right_on='Title',how='inner')
movie_df_final = df1.merge(mojo,left_on='Title',right_on='Title',how='inner')

movie_df_final.drop(columns=['Unnamed: 0_x','year_y','original_release_date','runtime','Unnamed: 0_y','year','directors'],inplace=True)
movie_df_final = movie_df_final.rename(columns={'year_x':'Year','audience_rating':'tomato_audience_rating','audience_count':'tomato_audience_vote_count'})
movie_df_final['Domestic_Lifetime_Gross'] = movie_df_final['Domestic_Lifetime_Gross'].str.replace('-','0')
movie_df_final['Domestic_Lifetime_Gross'] = movie_df_final['Domestic_Lifetime_Gross'].astype(int)
print(movie_df_final.columns)
print(movie_df_final.shape)
print(movie_df_final.head())
print(movie_df_final.dtypes)
print(movie_df_final.describe())





Index(['imdb_title_id', 'Title', 'Alternative Title', 'director', 'Year',
       'date_published', 'Runtime', 'imdb_avg_vote', 'imdb_total_votes',
       'imdb_us_voters_rating', 'imdb_us_voters_votes',
       'imdb_non_us_voters_rating', 'imdb_non_us_voters_votes',
       'imdb_females__avg_vote', 'imdb_females_votes', 'imdb_males_avg_vote',
       'imdb_males_votes', 'Metascore', 'tomatometer_rating',
       'tomatometer_count', 'tomato_audience_rating',
       'tomato_audience_vote_count', 'Worldwide_Lifetime_Gross',
       'Domestic_Lifetime_Gross', 'Foreign_Lifetime_Gross'],
      dtype='object')
  imdb_title_id                                        Title  \
0     tt2226597                      The Mountain Between Us   
1     tt2231461                                      Rampage   
2     tt2239822  Valerian and the City of a Thousand Planets   
3     tt2249081                                 The Forgiven   
4     tt2250912                       Spider-Man: Homecoming   

      

In [87]:
import recordlinkage

print(rotten.shape)
indexer = recordlinkage.Index()
indexer.block('Title')
pairs = indexer.index(df,rotten)
compare_cl = recordlinkage.Compare()
compare_cl.exact('year','year')
compare_cl.string('Title','Title',threshold=0.85)
compare_cl.string('Alternative Title','Title',threshold=0.70)
compare_cl.string('director','directors',threshold=0.85)
pot_matches = compare_cl.compute(pairs,df,rotten)
matches = pot_matches[pot_matches.sum(axis=1)>=2]
print(matches)


#z=0
#for i in df['Title']:
#    for j in mojo['title']:
#        Ratio = fuzz.ratio(i.lower(),j.lower())
#        if Ratio > 80 :
#            z += 1
#print (z)
        

(1751, 10)
           0    1    2    3
0    1496  1  1.0  0.0  1.0
1    1085  1  1.0  0.0  1.0
2    1667  1  1.0  0.0  1.0
3    1387  0  1.0  0.0  1.0
4    1216  1  1.0  1.0  1.0
...       ..  ...  ...  ...
1656 103   0  1.0  0.0  1.0
1659 485   1  1.0  0.0  1.0
1668 1554  1  1.0  1.0  1.0
1669 731   1  1.0  1.0  1.0
1671 1356  1  1.0  1.0  1.0

[841 rows x 4 columns]
